In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import streamlit as st

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


import kaggle

In [10]:
kaggle.api.authenticate()
dataset_url = 'ravirajsinh45/real-life-industrial-dataset-of-casting-product'
kaggle.api.dataset_download_files(dataset_url, unzip=True)


Dataset URL: https://www.kaggle.com/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product


In [11]:
train_generator = ImageDataGenerator(rotation_range = 360,
                                     width_shift_range = 0.05,
                                     height_shift_range = 0.05,
                                     shear_range = 0.05,
                                     zoom_range = 0.05,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     brightness_range = [0.75, 1.25],
                                     rescale = 1./255,
                                     validation_split = 0.2)

In [21]:
# get data 
#menentukan satu set nilai lain untuk parameter 'flow_from_directory':
IMAGE_DIR = "./casting_data/casting_data/"
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 64
SEED_NUMBER = 123

gen_args = dict(target_size = IMAGE_SIZE,
                color_mode = "grayscale",
                batch_size = BATCH_SIZE,
                class_mode = "binary",
                classes = {"ok_front": 0, "def_front": 1},
                seed = SEED_NUMBER)

train_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + "train",
                                        subset = "training", shuffle = True, **gen_args)
validation_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + "train",
                                        subset = "validation", shuffle = True, **gen_args)

Found 5307 images belonging to 2 classes.
Found 1326 images belonging to 2 classes.


In [22]:
mapping_class = {0: "normal", 1: "defect"}
mapping_class

{0: 'normal', 1: 'defect'}

In [23]:
def visualizeImageBatch(dataset, title):
    images, labels = next(iter(dataset))
    images = images.reshape(BATCH_SIZE, *IMAGE_SIZE)
    fig, axes = plt.subplots(8, 8, figsize=(16,16))

    for ax, img, label in zip(axes.flat, images, labels):
        ax.imshow(img, cmap = "gray")
        ax.axis("off")
        ax.set_title(mapping_class[label], size = 20)

    plt.tight_layout()
    fig.suptitle(title, size = 30, y = 1.05, fontweight = "bold")
    plt.show()
    
    return images

In [24]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, strides=2, activation="relu", input_shape=IMAGE_SIZE + (1, )))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=16, kernel_size=3, strides=2, activation="relu"))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(1, activation="sigmoid"))

2024-05-27 21:11:16.725136: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 149, 149, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 36, 36, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 5184)              0         
                                                                 
 dense (Dense)               (None, 128)               6

In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [27]:
checkpoint = ModelCheckpoint('model/cnn_casting_inspection_model.keras', verbose=1, save_best_only=True, monitor='val_loss', mode='min')

model.fit(train_dataset, validation_data=validation_dataset, batch_size=16, epochs=10, callbacks=[checkpoint], verbose=1)


Epoch 1/10


2024-05-27 21:11:32.566952: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


83/83 [==============================] - ETA: 0s - loss: 0.6761 - accuracy: 0.5775

2024-05-27 21:13:49.337806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 0.61928, saving model to model/cnn_casting_inspection_model.keras
83/83 [==============================] - 158s 2s/step - loss: 0.6761 - accuracy: 0.5775 - val_loss: 0.6193 - val_accuracy: 0.6463
Epoch 2/10
83/83 [==============================] - ETA: 0s - loss: 0.6053 - accuracy: 0.6593
Epoch 2: val_loss improved from 0.61928 to 0.53944, saving model to model/cnn_casting_inspection_model.keras
83/83 [==============================] - 163s 2s/step - loss: 0.6053 - accuracy: 0.6593 - val_loss: 0.5394 - val_accuracy: 0.7391
Epoch 3/10
83/83 [==============================] - ETA: 0s - loss: 0.5057 - accuracy: 0.7398
Epoch 3: val_loss improved from 0.53944 to 0.43658, saving model to model/cnn_casting_inspection_model.keras
83/83 [==============================] - 149s 2s/step - loss: 0.5057 - accuracy: 0.7398 - val_loss: 0.4366 - val_accuracy: 0.8122
Epoch 4/10
83/83 [==============================] - ETA: 0s - loss: 0.4256 - accuracy: 0.7969
Epoc

In [28]:
tf.keras.models.save_model(model,'/home/taufiqu/Capstone37')

2024-05-27 21:47:09.306242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2024-05-27 21:47:09.380997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2024-05-27 21:47:11.038871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2024-05-27 21:47:

INFO:tensorflow:Assets written to: /home/taufiqu/Capstone37/assets


INFO:tensorflow:Assets written to: /home/taufiqu/Capstone37/assets
